In [1]:
# ==========================================================
# ShuffleNetV2-x1.0 | CIFAR-10 | PyTorch | 5 Seeds | GPU Supported
# ==========================================================

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import random
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [2]:
# --------------------------
# Device setup
# --------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [3]:
# --------------------------
# Transformations & Dataloaders
# --------------------------
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)


100%|██████████| 170M/170M [00:05<00:00, 28.8MB/s]


In [4]:
# --------------------------
# Model Definition (moderate fine-tuning)
# --------------------------
class ShuffleNetV2_Model(nn.Module):
    def __init__(self):
        super(ShuffleNetV2_Model, self).__init__()
        self.model = torchvision.models.shufflenet_v2_x1_0(weights="IMAGENET1K_V1")

        
        for param in self.model.parameters():
            param.requires_grad = False
        for name, param in list(self.model.named_parameters())[-60:]:
            param.requires_grad = True

        # Replace final FC for 10 CIFAR-10 classes
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, 10)

    def forward(self, x):
        return self.model(x)

In [5]:
# --------------------------
# Training Function
# --------------------------
def train_model(model, trainloader, criterion, optimizer, epochs=4):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {running_loss/len(trainloader):.4f}")


In [6]:
# --------------------------
# Evaluation Function
# --------------------------
def evaluate_model(model, testloader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    return acc, prec, rec, f1

In [7]:
# --------------------------
# Multi-Seed Training
# --------------------------
seeds = [42, 123, 340, 777, 999]
results = []

for seed in seeds:
    print(f"\n=== Running for seed: {seed} (4 epochs) ===")
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    model = ShuffleNetV2_Model().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-4)

    train_model(model, trainloader, criterion, optimizer, epochs=4)
    acc, prec, rec, f1 = evaluate_model(model, testloader)
    results.append([acc, prec, rec, f1])
    print(f"Seed {seed} → Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")



=== Running for seed: 42 (4 epochs) ===
Downloading: "https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x1-5666bf0f80.pth


100%|██████████| 8.79M/8.79M [00:00<00:00, 62.7MB/s]


Epoch [1/4] - Loss: 0.7803
Epoch [2/4] - Loss: 0.3063
Epoch [3/4] - Loss: 0.2362
Epoch [4/4] - Loss: 0.1858
Seed 42 → Acc: 0.9179, Prec: 0.9182, Rec: 0.9179, F1: 0.9178

=== Running for seed: 123 (4 epochs) ===
Epoch [1/4] - Loss: 0.7896
Epoch [2/4] - Loss: 0.3071
Epoch [3/4] - Loss: 0.2348
Epoch [4/4] - Loss: 0.1838
Seed 123 → Acc: 0.9175, Prec: 0.9175, Rec: 0.9175, F1: 0.9173

=== Running for seed: 340 (4 epochs) ===
Epoch [1/4] - Loss: 0.7885
Epoch [2/4] - Loss: 0.3100
Epoch [3/4] - Loss: 0.2341
Epoch [4/4] - Loss: 0.1874
Seed 340 → Acc: 0.9179, Prec: 0.9185, Rec: 0.9179, F1: 0.9178

=== Running for seed: 777 (4 epochs) ===
Epoch [1/4] - Loss: 0.7782
Epoch [2/4] - Loss: 0.3049
Epoch [3/4] - Loss: 0.2313
Epoch [4/4] - Loss: 0.1848
Seed 777 → Acc: 0.9180, Prec: 0.9183, Rec: 0.9180, F1: 0.9180

=== Running for seed: 999 (4 epochs) ===
Epoch [1/4] - Loss: 0.7902
Epoch [2/4] - Loss: 0.3141
Epoch [3/4] - Loss: 0.2364
Epoch [4/4] - Loss: 0.1912
Seed 999 → Acc: 0.9157, Prec: 0.9163, Rec: 0.

In [8]:
# --------------------------
# Mean ± SD across Seeds
# --------------------------
results = np.array(results)
metrics = ["Accuracy", "Precision", "Recall", "F1-Score"]

print("\n=== Final Results (Mean ± SD across 5 seeds) ===")
for i, metric in enumerate(metrics):
    mean_val = results[:, i].mean()
    sd_val = results[:, i].std()
    print(f"{metric}: {mean_val:.4f} ± {sd_val:.4f}")


=== Final Results (Mean ± SD across 5 seeds) ===
Accuracy: 0.9174 ± 0.0009
Precision: 0.9178 ± 0.0008
Recall: 0.9174 ± 0.0009
F1-Score: 0.9173 ± 0.0009
